In [1]:
import os
print(os.getcwd())
#change working directory to the location of the data file
os.chdir('/mnt/d/Sajjad/08-2023/Python Code/Text Analytics Fall 2024/')
print(os.getcwd())

/mnt/c/Users/sahaider/AppData/Local/Programs/Microsoft VS Code
/mnt/d/Sajjad/08-2023/Python Code/Text Analytics Fall 2024


In [2]:
from transformers import pipeline
import pandas as pd

In [7]:
sentences = ["it is nice weather ", "it is cold ", "it is not very cold ", "not nice ", "very cold ", "cold ", "very nice weather ", "it is not nice cold at all"]

In [8]:
text = "it is not nice cold at all"

Using PLM (bert-finetune-sst2)

In [31]:
classifier = pipeline("text-classification")
outputs = classifier(text)
pd.DataFrame(outputs) 

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


,label,score
0,NEGATIVE,0.999766


In [ ]:
for s in sentences:
    outputs = classifier(s)
    print(s, outputs[0]['label'], outputs[0]['score'])

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import warnings
warnings.filterwarnings("ignore")
import textwrap

In [4]:
# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-1_8B-Chat", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen-1_8B-Chat-Int4", #"Qwen/Qwen-7B-Chat-Int4",
    device_map="auto",
    trust_remote_code=True
).eval()


Try importing flash-attention for faster inference...


In [5]:
#print model footprint
print(f"model footprint: {model.num_parameters() / 1e6:.2f}M")
#print memory taken by the model
print(f"model memory: {model.num_parameters() * 4 / 1e6:.2f}MB")

model footprint: 622.43M
model memory: 2489.72MB


Solving class example using Qwen LLM

In [6]:
prompt = f"""
Classify the last sentence using the following examples.

<sentence>: it is nice weather 
<sentiment>: positive
<sentence>: it is cold 
<sentiment>: negative
<sentence>: it is not very cold 
<sentiment>: positive
<sentence>: not nice 
<sentiment>: negative
<sentence>: very cold 
<sentiment>: negative
<sentence>: cold 
<sentiment>: negative
<sentence>: very nice weather 
<sentiment>: positive
<sentence>: it is not nice cold at all

"""
response, history = model.chat(tokenizer, prompt, history=None)
print(textwrap.fill(response, width=80))

The last sentence in the given examples is:  <sentiment>: not nice  <sentiment>:
cold  <sentiment>: very cold  <sentiment>: negative   These sentences express
different sentiments related to the weather. The first sentence has a positive
sentiment as "nice" is used, while the second sentence expresses a negative
sentiment with words like "cold", "not nice", and "very cold". The third
sentence shows that there's a mix of positive and negative sentiments as "not
nice" and "cold" are mentioned together. The fourth sentence indicates that the
weather is not pleasant at all with words like "nice" and "cold". The fifth
sentence also has a mixed sentiment, with words like "not nice" and "very cold".
Overall, the last sentence can be classified as "negative" because it contains
negative sentiments such as "cold" and "very cold."


Using Qwen to evaluate all the 8 comments

In [9]:
for i, s in enumerate(sentences):
    text = s
    print(i, ":", text)
    prompt = f"""
    What is the sentiment of the following text, which is delimited with triple backticks?

    Give your answer as either "positive" or "negative". If possible, also share the probability of your answer.
    Review text: '''{text}'''
    """
    response, history = model.chat(tokenizer, prompt, history=None)
    print(textwrap.fill(response, width=80))
    print("----------------------------------")

0 : it is nice weather 
The sentiment of the given text is positive. The word "nice" has a positive
connotation and suggests that the person who wrote it is enjoying the weather.
Probability: 100% (since the sentiment of the text is clearly positive)
----------------------------------
1 : it is cold 
The sentiment of the given text is negative.   Probability: The sentiment
analysis model may have been trained on limited data and might not be able to
accurately determine the sentiment of this text. Therefore, the probability
cannot be provided without more context and information about the training data
used for the sentiment analysis model.
----------------------------------
2 : it is not very cold 
The sentiment of the given text is negative. The words "not very cold" suggest
that the temperature is chilly, and the overall tone is negative.
----------------------------------
3 : not nice 
The sentiment of the text can be classified as negative because the words "not
nice" indicate a n

Loading IMDB dataset

In [10]:
# load the dataset
with open('corpus.txt','r',encoding="utf8") as f:
    document = f.readlines()
f.close()

labels, texts = [], []
for line in document:
    content = line.split()
    label = content[0]
    labels.append(label[-1])
    texts.append(" ".join(content[1:]))

print(len(labels), len(texts))

10000 10000


In [11]:
texts[0:5]

['Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^',
 "The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny.",
 'Amazing!: This soundtrack is my favorite music of all

In [12]:
labels[0:10]

['2', '2', '2', '2', '2', '2', '1', '2', '2', '2']

Preparing BoW using CountVectorizer and TfidfVectorizer

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [14]:
cnt_vectorizer = CountVectorizer()
features = cnt_vectorizer.fit_transform(texts)
features_nd = features.toarray()
features_nd.shape

(10000, 31627)

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)

In [17]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [18]:
clf1 = MultinomialNB()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

0.846

In [ ]:
#using countvectorize with trigram and remove stop words


In [19]:
cnt_vectorizer = CountVectorizer(ngram_range=(1, 2),max_features=100)
features = cnt_vectorizer.fit_transform(texts)
features_nd = features.toarray()
print(features_nd.shape)
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)
clf1 = MultinomialNB()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

(10000, 100)


0.7156

In [20]:
cnt_vectorizer = CountVectorizer(ngram_range=(1, 2),max_features=200)
features = cnt_vectorizer.fit_transform(texts)
features_nd = features.toarray()
print(features_nd.shape)
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)
clf1 = MultinomialNB()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

(10000, 200)


0.7384

In [21]:
cnt_vectorizer = CountVectorizer(ngram_range=(1, 2),max_features=500)
features = cnt_vectorizer.fit_transform(texts)
features_nd = features.toarray()
print(features_nd.shape)
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)
clf1 = MultinomialNB()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

(10000, 500)


0.8024

In [22]:
cnt_vectorizer = CountVectorizer(ngram_range=(1, 2),max_features=2000)
features = cnt_vectorizer.fit_transform(texts)
features_nd = features.toarray()
print(features_nd.shape)
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)
clf1 = MultinomialNB()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

(10000, 2000)


0.8264

In [23]:
cnt_vectorizer = CountVectorizer(ngram_range=(1, 5),max_features=2000)
features = cnt_vectorizer.fit_transform(texts)
features_nd = features.toarray()
print(features_nd.shape)
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)
clf1 = MultinomialNB()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

(10000, 2000)


0.8228

In [24]:
cnt_vectorizer = CountVectorizer(ngram_range=(1, 3),max_features=5000)
features = cnt_vectorizer.fit_transform(texts)
features_nd = features.toarray()
print(features_nd.shape)
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)
clf1 = MultinomialNB()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

(10000, 5000)


0.8432

In [25]:
tfidf_vectorizer = TfidfVectorizer()#(ngram_range=(1, 3),max_features=5000)
features = tfidf_vectorizer.fit_transform(texts)
features_nd = features.toarray()
print(features_nd.shape)
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)
clf1 = MultinomialNB()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

(10000, 31627)


0.838

In [26]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3),max_features=5000)
features = tfidf_vectorizer.fit_transform(texts)
features_nd = features.toarray()
print(features_nd.shape)
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)
clf1 = MultinomialNB()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

(10000, 5000)


0.8456

In [27]:
clf1 = RandomForestClassifier()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

0.8336

In [28]:
clf1 = KNeighborsClassifier()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

0.7396

In [29]:
X_train2, X_test2, y_train, y_test  = train_test_split(texts, labels, train_size=0.75,random_state=1234)

Evaluating all 2500 test records using PLM (bert-finetuned-sst)

This may take upto 2-3 minutes on your local machine

In [32]:
bert_pred = []
for s in X_test2:
    outputs = classifier(s)
    bert_pred.append(outputs[0]['label'])

In [33]:
#map 'Positive' to 2 and 'Negative' to 1 in bert_pred list 
bert_pred2 = []
for s in bert_pred:
    if s == 'POSITIVE':
        bert_pred2.append('2')
    else:
        bert_pred2.append('1')
accuracy_score(y_test, bert_pred2)

0.9016

In [ ]:
# take top 100 records from X_test2
X_test2a = X_test[0:100]

Qwen and Zephyr can't be done on local machines (specially laptops) and should be done on Cloud